In [ ]:
import pandas as pd
import numpy as np
health = pd.read_csv('data/cleaned_trainingdata.csv')
health = health.set_index('person_id_syn')

In [ ]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
health.head()

,betos_d1c_ind,betos_d1d_ind,betos_m1b_ind,betos_m2c_ind,betos_m5b_ind,betos_m5c_ind,betos_m5d_ind,betos_o1a_ind,betos_o1b_ind,betos_o1e_ind,...,count_men,count_mus,count_ner,count_rar,count_res,count_rsk,count_skn,count_sns,count_sor,count_vco
person_id_syn,,,,,,,,,,,,,,,,,,,,,
0002MOb79ST17bLYAe46eIc2,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.945441,0.000000,0.0,0.177619,0.316758,0.000000,0.479521,0.087879,0.288975
0004cMOS6bTLf34Y7AIca8f3,0,0,1,0,0,1,1,0,0,0,...,0.356489,0.389335,0.084335,0.0,0.000000,0.673225,0.089573,0.349359,0.173064,0.268385
000536M9O3ST98LaYaeA29Ia,0,0,1,0,0,1,1,1,0,1,...,0.000000,0.833333,0.000000,0.0,0.000000,0.416667,0.083333,2.333334,0.250000,0.416667
0009bMO9SfTLYe77A51I4ac3,0,0,1,0,0,0,1,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.166667,0.500000,0.000000,0.000000,0.083333,0.166667
000M7OeS66bTL8bY89Aa16Ie,0,0,1,0,0,0,0,0,0,1,...,0.000000,0.000000,0.250000,0.0,0.666667,0.583333,0.000000,0.000000,0.000000,0.333333


In [ ]:
Y = health['transportation_issues']
X = health.drop('transportation_issues', axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_holdout_test, y_train, y_holdout_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42, sampling_strategy = 1)
X_train_under50, y_train_under50 = rus.fit_sample(X_train, y_train)

rus = RandomUnderSampler(random_state=42, sampling_strategy = (2/3))
X_train_under40, y_train_under40 = rus.fit_sample(X_train, y_train)

rus = RandomUnderSampler(random_state=42, sampling_strategy = (3/7))
X_train_under30, y_train_under30 = rus.fit_sample(X_train, y_train)

rus = RandomUnderSampler(random_state=42, sampling_strategy = 0.25)
X_train_under20, y_train_under20 = rus.fit_sample(X_train, y_train)

In [ ]:
from collections import Counter
counter = Counter(y_train_under50)
print(counter)

Counter({0: 7698, 1: 7698})


In [ ]:
import imblearn
oversample = SMOTE(sampling_strategy = 1)
X_train_over50, y_train_over50 = oversample.fit_resample(X_train, y_train)

oversample = SMOTE(sampling_strategy = (2/3))
X_train_over40, y_train_over40 = oversample.fit_resample(X_train, y_train)

oversample = SMOTE(sampling_strategy = (3/7))
X_train_over30, y_train_over30 = oversample.fit_resample(X_train, y_train)

oversample = SMOTE(sampling_strategy = 0.25)
X_train_over20, y_train_over20 = oversample.fit_resample(X_train, y_train)

In [ ]:
from collections import Counter
counter = Counter(y_train_over50)
print(counter)

Counter({0: 44481, 1: 44481})


In [ ]:
over = SMOTE(sampling_strategy=0.3)
under = RandomUnderSampler(sampling_strategy=1)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
X_train_OverUnder, y_train_OverUnder = pipeline.fit_resample(X_train, y_train)

In [ ]:
counter = Counter(y_train_OverUnder)
print(counter)

Counter({0: 13344, 1: 13344})
